In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import pickle
import matplotlib.pyplot as plt

C:\Users\Saleem\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torchvision.transforms as transforms

# Hyper Parameters
num_epochs = 150
batch_size = 1000
learning_rate = 0.001

# Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.247, 0.2434, 0.2615)),
])







# CIFAR-10 Dataset
train_dataset = dsets.CIFAR10(root='./data/',
                               train=True,
                               transform=transform,
                               download=True)



test_dataset = dsets.CIFAR10(root='./data/',
                              train=False,
                              transform=transform,
                              download=True)



Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)




test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 96, kernel_size=3, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(2))

        self.fc = nn.Linear(4*4*96, 10)
        self.dropout = nn.Dropout(p=0.5)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        out = self.fc(out)
        return self.logsoftmax(out)

In [5]:
cnn = CNN()

if torch.cuda.is_available():
    cnn = cnn.cuda()

# convert all the weights tensors to cuda()
# Loss and Optimizer



criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)


print('number of parameters: ', sum(param.numel() for param in cnn.parameters()))

number of parameters:  48490


In [6]:
# cnn(next(iter(train_loader))[0])
cnn.train()

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1536, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (logsoftmax): 

In [9]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # print(i, end=' ')
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        # Forward + Backward + Optimize
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 25 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                   %(epoch+1, num_epochs, i+1,
                     len(train_dataset)//batch_size, loss.data))

Epoch [1/150], Iter [25/50] Loss: 12.6502
Epoch [1/150], Iter [50/50] Loss: 11.2915
Epoch [2/150], Iter [25/50] Loss: 10.3801
Epoch [2/150], Iter [50/50] Loss: 9.4361
Epoch [3/150], Iter [25/50] Loss: 8.0566
Epoch [3/150], Iter [50/50] Loss: 7.0727
Epoch [4/150], Iter [25/50] Loss: 6.2709
Epoch [4/150], Iter [50/50] Loss: 6.4896
Epoch [5/150], Iter [25/50] Loss: 5.5985
Epoch [5/150], Iter [50/50] Loss: 5.0500
Epoch [6/150], Iter [25/50] Loss: 4.3324
Epoch [6/150], Iter [50/50] Loss: 4.2054
Epoch [7/150], Iter [25/50] Loss: 4.3983
Epoch [7/150], Iter [50/50] Loss: 3.9462
Epoch [8/150], Iter [25/50] Loss: 3.6722
Epoch [8/150], Iter [50/50] Loss: 3.5739
Epoch [9/150], Iter [25/50] Loss: 3.2528
Epoch [9/150], Iter [50/50] Loss: 3.0840
Epoch [10/150], Iter [25/50] Loss: 3.3061
Epoch [10/150], Iter [50/50] Loss: 3.0210
Epoch [11/150], Iter [25/50] Loss: 2.8836
Epoch [11/150], Iter [50/50] Loss: 2.9190
Epoch [12/150], Iter [25/50] Loss: 2.6833
Epoch [12/150], Iter [50/50] Loss: 2.9363
Epoch [

KeyboardInterrupt: 

In [ ]:
cnn.eval()
correct = 0
total = 0

for images, labels in test_loader:
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()

    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))